In [ ]:
np.sum(np.diff(x) * (y[:-1] + np.diff(y)/2))

In [1]:
import numpy as np

In [2]:
class Preferences(object):
    def __init__(self):
        self.end = 0
        self.endvalue = 0
        self.id = 0
        self.start = 0
        self.startvalue = 0

In [3]:
#row is agent, column is slice
preferences = [[Preferences() for i in range(3)] for j in range(3)]

In [4]:
preferences[0][0].end = 1
preferences[0][0].endvalue = 3.7
preferences[0][0].id = 1
preferences[0][0].start = 0
preferences[0][0].startvalue = 5.8

In [5]:
preferences[0][1].end = 2
preferences[0][1].endvalue = 5.5
preferences[0][1].id = 2
preferences[0][1].start = 1
preferences[0][1].startvalue = 5.5

In [6]:
preferences[0][2].end = 3
preferences[0][2].endvalue = 7
preferences[0][2].id = 3
preferences[0][2].start = 2
preferences[0][2].startvalue = 1.6

In [7]:
preferences[1][0].end = 1
preferences[1][0].endvalue = 7.3
preferences[1][0].id = 4
preferences[1][0].start = 0
preferences[1][0].startvalue = 3

In [8]:
preferences[1][1].end = 2
preferences[1][1].endvalue = 7.5
preferences[1][1].id = 5
preferences[1][1].start = 1
preferences[1][1].startvalue = 3

In [9]:
preferences[1][2].end = 3
preferences[1][2].endvalue = 0
preferences[1][2].id = 6
preferences[1][2].start = 2
preferences[1][2].startvalue = 10

In [10]:
preferences[2][0].end = 1
preferences[2][0].endvalue = 4.3
preferences[2][0].id = 7
preferences[2][0].start = 0
preferences[2][0].startvalue = 9.4

In [11]:
preferences[2][1].end = 3
preferences[2][1].endvalue = 7.6
preferences[2][1].id = 8
preferences[2][1].start = 1
preferences[2][1].startvalue = 7.6

In [12]:
preferences[2][2] = None

In [13]:
cakeSize = 3

In [14]:
#prefs, normalizationConstant = oneLipschitz(preferences, cakeSize)
preferences = oneLipschitz(preferences, cakeSize)

NameError: name 'oneLipschitz' is not defined

1.0000000000000002

In [45]:
epsilon = 0.0025

Preprocessing

Hungriness:

In [ ]:
def branzeiNisan(prefs, cakeSize, epsilon):
    preferences = oneLipschitz(prefs, cakeSize)
    alphaLowerBound = computeEquipartitionFourAgents(prefs, epsilon)
    if checkEnvyFree(prefs, alphaLowerBound) == True:
        return division(prefs, alphaLowerBound)
    alphaUpperBound = 1
    while abs(alphaUpperBound - alphaLowerBound) > ((epsilon/10)**4)/12:
        alpha = (alphaLowerBound + alphaUpperBound) / 2
        if checkConditions(prefs, alpha) == True:
            alphaLowerBound = alpha
        else:
            alphaUpperBound = alpha
    return division(prefs, alphaLowerBound)
        

In [102]:
cutBounds = findLeftCutBounds(preferences, epsilon)

In [103]:
left = cutBounds.midpoint()
valueQuery(0,preferences, 0, left, epsilon) / valueQuery(0,preferences, left, 1, epsilon)

0.5000920435826642

In [100]:
cutBounds = findRightCutBounds(preferences, epsilon)

In [101]:
right = cutBounds.midpoint()
valueQuery(0,preferences, right, 1, epsilon) / valueQuery(0,preferences, 0, right, epsilon)

0.5001954804512858

In [39]:
check = Bounds(0.6344324880074275, 0.6344324880074286)

In [38]:
check.midpoint()

0.634432488007428

Ignore Below.

In [15]:
def computeEquipartitionThreeAgents(prefs, epsilon):
    leftCutBounds = findLeftCutBounds(prefs, epsilon)
    rightCutBounds = findRightCutBounds(prefs, epsilon)
    equipartition = \
        exactEquipartitionCutsThreeAgents(prefs, leftCutBounds, rightCutBounds)

In [65]:
def findLeftCutBounds(prefs, epsilon):
    leftCutBounds = Bounds(0, 1)
    while abs(leftCutBounds.upper - leftCutBounds.lower) - epsilon > 1e-10:
        leftCutBounds = leftCutBoundsUpdate(prefs, leftCutBounds, epsilon)
    return leftCutBounds
            

In [59]:
def leftCutBoundsUpdate(prefs, leftCutBounds, epsilon):
    leftCut = leftCutBounds.midpoint()
    segmentOneValue = valueQuery(0, prefs, 0, leftCut, epsilon)
    rightCut = cutQuery(0, prefs, leftCut, segmentOneValue, 
                        epsilon, endCut = True)
    leftCutBounds = leftAdjustmentThreeAgent(prefs, leftCutBounds, 
                                             rightCut, segmentOneValue, 
                                             epsilon)
    return leftCutBounds

In [71]:
def findRightCutBounds(prefs, epsilon):
    rightCutBounds = Bounds(0, 1)
    while abs(rightCutBounds.upper - rightCutBounds.lower) - epsilon > 1e-10:
        rightCutBounds = rightCutBoundsUpdate(prefs, rightCutBounds, epsilon)
    return rightCutBounds

In [87]:
def rightCutBoundsUpdate(prefs, rightCutBounds, epsilon):
    rightCut = rightCutBounds.midpoint()
    segmentThreeValue = valueQuery(0, prefs, rightCut, 1, epsilon)
    leftCut = cutQuery(0, prefs, rightCut, segmentThreeValue, 
                        epsilon, endCut = False)
    rightCutBounds = rightAdjustmentThreeAgent(prefs, rightCutBounds, 
                                               leftCut, segmentThreeValue, 
                                               epsilon)
    return rightCutBounds

In [ ]:
def exactEquipartitionCutsThreeAgents(prefs, leftCutBounds, rightCutBounds):
    leftCut = equipartitionLeftCutThreeAgents(prefs, leftCutBounds)
    rightCut= equipartitionLeftCutThreeAgents(prefs, rightCutBounds)
    equipartition = ThreeAgentPortion(leftCut, rightCut)
    return equipartition
    

In [72]:
#Must add functionality that returns if there is no cut.
def cutQuery(agent, prefs, initialCut, value, epsilon, endCut = True):
    if endCut == True:
        queriedCut = endCutQuery(agent, prefs, initialCut, value, epsilon)
    else:
        queriedCut = startCutQuery(agent, prefs, initialCut, value, epsilon)
    return queriedCut

In [57]:
def endCutQuery(agent, prefs, start, value, epsilon):
    endCutBounds = Bounds(start, 1)
    while abs(endCutBounds.upper - endCutBounds.lower) > 1e-15:
        endCutBounds = endCutBoundsUpdate(prefs, start, endCutBounds, 
                                          value, epsilon)
    endCut = endCutBounds.midpoint()
    return endCut

In [66]:
def endCutBoundsUpdate(prefs, start, endCutBounds, 
                       value, epsilon):
    endCut = endCutBounds.midpoint()
    queriedValue = valueQuery(0, prefs, start, endCut, epsilon)
    if queriedValue <= value:
        endCutBounds.lower  = endCut
    if queriedValue > value:
        endCutBounds.upper = endCut
    return endCutBounds

In [81]:
def startCutQuery(agent, prefs, end, value, epsilon):
    startCutBounds = Bounds(0, end)
    while abs(startCutBounds.upper - startCutBounds.lower) > 1e-15:
        startCutBounds = startCutBoundsUpdate(prefs, end, startCutBounds, 
                                              value, epsilon)
    startCut = startCutBounds.midpoint()
    return startCut

In [99]:
def startCutBoundsUpdate(prefs, end, startCutBounds, 
                         value, epsilon):
    startCut = startCutBounds.midpoint()
    queriedValue = valueQuery(0, prefs, startCut, end, epsilon)
    if queriedValue <= value:
        startCutBounds.upper  = startCut
    if queriedValue > value:
        startCutBounds.lower = startCut
    return startCutBounds

In [55]:
def leftAdjustmentThreeAgent(prefs, leftCutBounds, rightCut,
                             segmentOneValue, epsilon):
    leftCut = leftCutBounds.midpoint()
    segmentThreeValue = valueQuery(0, prefs, rightCut, 1, epsilon)
    if segmentThreeValue >= segmentOneValue:
        leftCutBounds.lower = leftCut
    if segmentThreeValue < segmentOneValue:
        leftCutBounds.upper = leftCut
    return leftCutBounds

In [67]:
def rightAdjustmentThreeAgent(prefs, rightCutBounds, leftCut, 
                              segmentThreeValue, epsilon):
    rightCut = rightCutBounds.midpoint()
    segmentOneValue = valueQuery(0, prefs, 0, leftCut, epsilon)
    if segmentOneValue >= segmentThreeValue:
        rightCutBounds.upper = rightCut
    if segmentOneValue < segmentThreeValue:
        rightCutBounds.lower = rightCut
    return rightCutBounds

In [20]:
#Remember to scale by L at end.
def oneLipschitz(prefs, cakeSize):
    prefs = changeBounds(prefs, cakeSize)
    #prefs, normalizationConstants = normalization(prefs)
    prefs = normalization(prefs)
    return prefs#, normalizationConstants

In [21]:
def changeBounds(prefs, cakeSize):
    agents = np.shape(prefs)[0]
    segments = np.shape(prefs)[1]
    for i in range(agents):
        for j in range(segments):
            if prefs[i][j] is not None:
                prefs[i][j].start /= cakeSize
                prefs[i][j].end /= cakeSize
    return prefs
        

In [22]:
'''
Ensures that each agents values the
'''
def normalization(prefs):
    agents = np.shape(prefs)[0]
    segments = np.shape(prefs)[1]
    normalizationConstants = np.zeros(agents)
    for i in range(agents):
        normalizationConstants[i] = valueQueryInitial(i, prefs, 0, 1)
        for j in range(segments):
            if prefs[i][j] is not None:
                prefs[i][j].startvalue /= normalizationConstants[i]
                prefs[i][j].endvalue /= normalizationConstants[i]
    return prefs#, normalizationConstants
        

In [23]:
'''
Linear interpolation on epsilon grid.
'''
def valueQuery(agent, prefs, start, end, epsilon):
    startBounds, endBounds = piecewiseLinearBounds(start, end, epsilon)
    
    if startBounds.upper - start >= end - endBounds.lower:
        return valueQueryVariantOne(agent, prefs, start, end, epsilon)
    if startBounds.upper - start < end - endBounds.lower:
        return valueQueryVariantTwo(agent, prefs, start, end, epsilon)

In [24]:
def valueQueryVariantOne(agent, prefs, start, end, epsilon):
    startBounds, endBounds = piecewiseLinearBounds(start, end, epsilon)
    queries = intermediateQueriesVariantOne(agent, prefs, startBounds, 
                                            endBounds, epsilon)
    componentOne = (((startBounds.upper - start) - (end - endBounds.lower)) / 
                    epsilon) * queries[0]
    componentTwo = ((end - endBounds.lower) / epsilon) * queries[1]
    componentThree = ((start - startBounds.lower) / epsilon) * queries[2]
    return componentOne + componentTwo + componentThree

In [25]:
def valueQueryVariantTwo(agent, prefs, start, end, epsilon):
    startBounds, endBounds = piecewiseLinearBounds(start, end, epsilon)
    queries = intermediateQueriesVariantTwo(agent, prefs, startBounds, 
                                            endBounds, epsilon)
    componentOne = (((end - endBounds.lower) - (startBounds.upper - start)) / 
                    epsilon) * queries[0]
    componentTwo = ((startBounds.upper - start) / epsilon) * queries[1]
    componentThree = ((endBounds.upper - end) / epsilon) * queries[2]
    return componentOne + componentTwo + componentThree

In [26]:
def piecewiseLinearBounds(start, end, epsilon):
    if start != 1:
        start_lower_bound = (start // epsilon) * epsilon
        start_upper_bound = startLowerBound + epsilon 
    else:
        start_lower_bound = 1 - epsilon
        startUpperBound = 1
    if end != 1:
        endLowerBound = (end // epsilon) * epsilon
        endUpperBound = endLowerBound + epsilon
    else:
        endLowerBound = 1 - epsilon
        endUpperBound = 1

    startBounds = Bounds(startLowerBound, startUpperBound)
    endBounds = Bounds(endLowerBound, endUpperBound)

    return startBounds, endBounds

In [27]:
def intermediate_queries_variant_one(agent, prefs, start_bounds, 
                                     end_bounds, epsilon):
    query_one = valueQueryHungry(agent, prefs, start_bounds.lower, 
                                end_bounds.lower, epsilon)
    query_two = valueQueryHungry(agent, prefs, start_bounds.lower, 
                                end_bounds.upper, epsilon)
    query_three = valueQueryHungry(agent, prefs, start_bounds.upper, 
                                  end_bounds.lower, epsilon)
    return query_one, query_two, query_three

In [28]:
def intermediate_queries_variant_two(agent, prefs, start_bounds, 
                                     end_bounds, epsilon):
    query_one = value_query_hungry(agent, prefs, start_bounds.upper, 
                                   end_bounds.upper, epsilon)
    query_two = value_query_hungry(agent, prefs, start_bounds.lower, 
                                   end_bounds.upper, epsilon)
    query_three = value_query_hungry(agent, prefs, start_bounds.upper, 
                                     end_bounds.lower, epsilon)
    return query_one, query_two, query_three

In [29]:
def value_query_hungry(agent, prefs, start, end, epsilon):
    initial_value = value_query_initial(agent, prefs, start, end)
    value = initial_value/2 + epsilon * (end - start)
    return value

In [30]:
def value_query_initial(agent, prefs, start, end):
    check_valid_bounds(start,end)
    if end <= start:
        value = 0
    else:
        value = one_sided_query(agent, prefs, end) - \
                one_sided_query(agent, prefs, start)
    return value
    

In [31]:
def check_valid_bounds(start,end):
    assert (start >= 0 and start <= 1 and end >= 0 and end <= 1), \
        "invalid bounds. start and end should be between 0 and 1."

In [32]:
def one_sided_query(agent, prefs, end):
    segments = np.size(prefs[agent])
    value = 0
    for i in range(segments):
        if end > prefs[agent][i].end:
            value += find_full_value(prefs[agent][i])
        if end <= prefs[agent][i].end:
            value += find_partial_value(prefs[agent][i], end)
            return value

In [33]:
def find_full_value(segment):
    segment_width = segment.end - segment.start
    if segment.startvalue == segment.endvalue:
        value = segment.startvalue * segment_width
        return value
    if segment.startvalue != segment.endvalue:
        value = 0.5 * (segment.startvalue + segment.endvalue) * segment_width
        return value

In [34]:
def find_partial_value(segment, partial_segment_end):
    segment_width = partial_segment_end - segment.start
    partial_segment_end_value = interpolate(segment, partial_segment_end)
    if segment.startvalue == segment.endvalue:
        value = segment.startvalue * segment_width
        return value
    if segment.startvalue != segment.endvalue:
        value = 0.5 * (segment.startvalue + partial_segment_end_value) * \
                segment_width
        return value

In [35]:
'''
Uses the linear interpolation formula to find the associated y-coordinate for a 
given x-coordinate on a line between two points with known x and y coordinates.
'''
def interpolate(segment, x_coordinate):
    if segment.startvalue == segment.endvalue:
        y_coordinate = segment.startvalue
        return y_coordinate
    if segment.startvalue != segment.endvalue:
        y_coordinate = ((segment.startvalue * (segment.end - x_coordinate) +
                         segment.endvalue * (x_coordinate - segment.start)) / 
                        (segment.end - segment.start))                
        return y_coordinate

In [36]:
class Bounds:
    def __init__(self, lower, upper):
        self.lower = lower
        self.upper = upper

    def midpoint(self):
        return (self.lower + self.upper) / 2


In [ ]:
class ThreeAgentPortion(left_cut, right_cut):
    def __init__(self, left_cut, right_cut):
        self.left = left_cut
        self.right = right_cut